In [95]:
from datetime import datetime
import pandas as pd
import numpy as np

In [96]:
from datetime import date

def pretvori_datum_v_datetime(string_datuma):
    """
    Funkcija, ki obe vrsti datuma pretvori v ustretno datetime obliko (datum_izida in datum all_time_high/low)
    """
    
    # Za vsak slučaj
    if not string_datuma:
        return None
    
    # Ločimo primera 'Apr 29, 2022' in '2022-04-29'
    if "," in string_datuma:
        return datetime.strptime(string_datuma, "%b %d, %Y")
    else:
        return datetime.strptime(string_datuma, "%Y-%m-%d")
    
print(pretvori_datum_v_datetime("Apr 9, 2022"))
print(pretvori_datum_v_datetime("2022-04-09"))

2022-04-09 00:00:00
2022-04-09 00:00:00


In [97]:
def popravi_cardtype_aftermath(seznam_cardtypa):
    """
    Popravimo seznam cardtypa, ki se pojavi zaradi nenavadnega/nekonsistentnega zapisa Aftermath kart na spletni strani
    """
    
    popravljen_seznam = []
    for tip in seznam_cardtypa:
        if tip not in popravljen_seznam and tip != "//":
            popravljen_seznam.append(tip)
    return popravljen_seznam

print(popravi_cardtype_aftermath(["Sorcery", "//", "Sorcery"]))

['Sorcery']


In [98]:
def popravi_subtype_adventure(seznam_subtypa):
    """
    Popravimo seznam subtypa, ki se pojavi zaradi nenavadnega/nekonsistentnega zapisa Adventure kart na spletni strani
    """
    
    popravljen_seznam = []
    for tip in seznam_subtypa:
        if tip == "//":
                break
        elif tip not in popravljen_seznam:
            popravljen_seznam.append(tip)
    return popravljen_seznam

print(popravi_subtype_adventure(['Beast', 'Noble', '//', 'Sorcery']))

['Beast', 'Noble']


In [99]:
problematicni_tipi = ["Token", "Emblem"]
def vsebovanost_problematicnih_karte_na_podlagi_seznama(seznam, seznam_problematicnih_elementov = problematicni_tipi):
    """
    Preverimo, ali določen seznam vsebuje problematične elemente
    """
    
    vsebuje_problem = False
    for element in seznam:
        if element in seznam_problematicnih_elementov:
            vsebuje_problem = True
            break
    return vsebuje_problem

print(vsebovanost_problematicnih_karte_na_podlagi_seznama(["Nissa"], problematicni_tipi))

False


In [109]:
color_pentagon = "wubrg"

def zavrti_cikel(cikel,smer_urinega = True):
    if smer_urinega:
        return cikel[1:] + cikel[0]
    else:
        return cikel[-1] + cikel[:-1]

def razdalja_med_crkama_v_ciklu(prva_crka, druga_crka, niz_barve, smer_urinega = True):
    if niz_barve[color_pentagon.index(prva_crka)] == druga_crka:
        return 0
    else:
        return 1 + razdalja_med_crkama_v_ciklu(prva_crka, druga_crka, zavrti_cikel(niz_barve, smer_urinega), smer_urinega)


print(razdalja_med_crkama_v_ciklu("u","w", color_pentagon, smer_urinega=True))

def prvi_string_je_v_drugem(niz1, niz2):
    for crka in niz1:
        if crka not in niz2:
            return False
    return True

def popravi_vrstni_red_barve(niz_barve):
    # Izkaže se, da je še najlažje ločiti funkcijo glede na to, koliko barv je vsebovanih
    # Celoten postopek določanja vrstnega reda je opisan tule: https://magic.wizards.com/en/articles/archive/ask-wizards-june-2004-2004-06-01
    for problematicen_znak in "(){}/p":
        if problematicen_znak in niz_barve:
            return niz_barve
    if len(niz_barve) < 2 or len(niz_barve) > 5:
        return niz_barve
    if len(niz_barve) == 5:
        return color_pentagon
    if len(niz_barve) == 2:
        transpozicija_niza = niz_barve[::-1]
        if razdalja_med_crkama_v_ciklu(niz_barve[0], niz_barve[1], color_pentagon) > 2:
            return transpozicija_niza
        else:
            return niz_barve
    if len(niz_barve) == 4:
        manjka_barva = color_pentagon[::]
        for crka in niz_barve:
            if crka not in manjka_barva:
                return niz_barve
            else:
                manjka_barva = manjka_barva.replace(crka, "")
        kopija_pentagona = color_pentagon[::]
        while kopija_pentagona[-1] != manjka_barva:
            kopija_pentagona = zavrti_cikel(kopija_pentagona)
        return kopija_pentagona.replace(manjka_barva, "")
    if len(niz_barve) == 3:
        
        if niz_barve in f"{color_pentagon},{zavrti_cikel(color_pentagon)},{zavrti_cikel(zavrti_cikel(color_pentagon))}":
            return niz_barve
        kopija_pentagona = color_pentagon[::]
        while not prvi_string_je_v_drugem(niz_barve, kopija_pentagona[::2]):
            kopija_pentagona = zavrti_cikel(kopija_pentagona)
        return kopija_pentagona[::2]
            
        
print(popravi_vrstni_red_barve("uw"))

def popravi_barvo_karte(niz_barve, niz_oracle_texta):
    """
    Ta funkcija vzame string barve, ki ga ustrezno popravi glede na oracle text + vrsntni red barv (UW -> WU)
    """
    
    # Tole moram še urediti, za nalogo samo niti ni tako bistveno, kot je iz osebnega interesa.
    pass

4
wu


# Zdaj hočemo narediti podatkovno bazo z merganjem dveh dataframov

In [101]:
osnovni_podatki_o_kartah = pd.read_csv("karte.csv")
dodatni_podatki_o_kartah = pd.read_csv("specificne_lastnosti_kart.csv")

podatkovna_baza = pd.merge(osnovni_podatki_o_kartah, dodatni_podatki_o_kartah)

%reset_selective -f osnovni_podatki_o_kartah
%reset_selective -f dodatni_podatki_o_kartah

Zdaj malo popravimo tabelo s funckijami, ki smo jih napisali.

In [112]:
podatkovna_baza["cardtype"] = podatkovna_baza["cardtype"].apply(popravi_cardtype_aftermath)
podatkovna_baza["cardtype"] = podatkovna_baza["cardtype"].apply(vsebovanost_problematicnih_karte_na_podlagi_seznama)
podatkovna_baza["subtype"] = podatkovna_baza["subtype"].apply(popravi_subtype_adventure)
#podatkovna_baza["povprecje_eu"] = podatkovna_baza["povprecje_eu"].str.replace(r"\u20ac", "€")

podatkovna_baza[["barva"]] = podatkovna_baza[["barva"]].apply(popravi_vrstni_red_barve)

podatkovna_baza.to_json("popolni_podatki_o_kartah.json", orient='records', indent=4)

TypeError: 'bool' object is not iterable

'wbg'